# Roboflow Video Inference

This colab notebook reflects [the Roboflow video inference repo](https://github.com/roboflow-ai/video-inference) but utilizes the pip package for inference calls instead of `CURL` commands.

Documentation for the Roboflow pip package can be found [via this link](https://docs.roboflow.com/python).

Video inference uses the following steps:
- break down videos to images using FFMPEG
- perform inference on each image and render a bounding box for each detected image
- stich images back together into a video format

# FFMPEG Set Up

### FFMPEG Installation

In [81]:
from IPython.display import clear_output
import os, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/yunooooo/gcct/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Installing Dependencies...", ty='twg')
os.system('pip install git+git://github.com/AWConant/jikanpy.git')
os.system('add-apt-repository -y ppa:jonathonf/ffmpeg-4')
os.system('apt-get update')
os.system('apt install mediainfo')
os.system('apt-get install ffmpeg')
clear_output()
print('Installation finished.')

Installation finished.


### FFMPEG Execution



#### Access video

Upload one video at a time for now to the `videos_to_infer` directory, video globs will result in videos being stiched together.

You can connect g-drive and use `cp` to copy a single file out of it or manually upload a video from your local device.

Regardless, make sure it lands in the `videos_to_infer` directory.

In [82]:
%cd /content/
!mkdir videos_to_infer
!mkdir inferred_videos
%cd videos_to_infer

/content
/content/videos_to_infer


#### Optional: Link your Google Drive to upload files to/from Google Drive


*   process outlined in the next 2 cells



In [ ]:
# OPTIONAL - link your g-drive to pull videos from
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# OPTIONAL - copy your videos from g-drive to /content/
!cp /content/gdrive/YOUR_VIDEO.mp4 /content/videos_to_images

#### Break down video frames into images

In [85]:
# break video down into images - UPDATE THE PATH TO THE FILE!
# Example: video file named 'Test.mp4'
# (cont'd) update path to: "/content/videos_to_infer/Test.mp4"
os.environ['inputFile'] = '/content/videos_to_infer/match_Trim.mp4'

# fps value: the number of frames to sample per second from the video
# higher value for fps: sample more frames
!ffmpeg  -hide_banner -loglevel error -i "$inputFile" -vf fps=10 "$inputFile_out%04d.png"

# Video Inference Section

## Roboflow PIP package installation

In [86]:
!pip3 install roboflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Imports


In [87]:
from roboflow import Roboflow
import json
from time import sleep
from PIL import Image, ImageDraw
import io
import base64
import requests
from os.path import exists
import os, sys, re, glob

## Initialization

- private api key found in Roboflow > YOUR_WORKSPACE > Roboflow API
- NOTE: this is your private key, not publishable key!

**Having trouble finding your API key, version number or project ID?** The [documentation's quick start section](https://docs.roboflow.com/python) demostrates how you can find these via the Roboflow platform UI.
* [Obtaining Your API Key](https://docs.roboflow.com/rest-api#obtaining-your-api-key)

In [88]:
# workspace code
from roboflow import Roboflow
import json

rf = Roboflow(api_key="6mmosvNzOLTgkbtR1Of4")
project = rf.workspace().project("raquette")
dataset = project.version(1)

# grab the model from that project's version
model = project.version(1).model
print(model)

loading Roboflow workspace...
loading Roboflow project...
{
  "id": "raquette/1",
  "name": "Raquette ",
  "version": "1",
  "classes": null,
  "overlap": 30,
  "confidence": 40,
  "stroke": 1,
  "labels": false,
  "format": "json",
  "base_url": "https://detect.roboflow.com/"
}


In [99]:
# HELPER FUNCTIONS BLOCK
def draw_boxes(box, x0, y0, img, class_name):
    # OPTIONAL - color map, change the key-values for each color to make the
    # class output labels specific to your dataset
    color_map = {
        "raquette_noire_Flore":"black",
        "class2":"red",
    }

    # get position coordinates
    bbox = ImageDraw.Draw(img) 

    bbox.rectangle(box, outline =color_map[class_name], width=5)
    bbox.text((x0, y0), class_name, fill='black', anchor='mm')

    return img

def save_with_bbox_renders(img):
    file_name = os.path.basename(img.filename)
    img.save('/content/inferred_videos/' + file_name)

## Execution

In [100]:
# perform inference on each image from the split up video

%cd /content/inferred_videos
!pwd
# glob config values
file_path = "/content/videos_to_infer/"
extention = ".png"

# glob files based on location and file format
globbed_files = sorted(glob.glob(file_path + '*' + extention))
#print(globbed_files)

for image in globbed_files:
  # INFERENCE
  predictions = model.predict(image).json()['predictions']
  newly_rendered_image = Image.open(image)

  # RENDER 
  # for each detection, create a crop and convert into CLIP encoding
  for prediction in predictions:
      print(prediction)
      # rip bounding box coordinates from current detection
      # note: infer returns center points of box as (x,y) and width, height
      # ----- but pillow crop requires the top left and bottom right points to crop
      x0 = prediction['x'] - prediction['width'] / 2
      x1 = prediction['x'] + prediction['width'] / 2
      y0 = prediction['y'] - prediction['height'] / 2
      y1 = prediction['y'] + prediction['height'] / 2
      box = (x0, y0, x1, y1)
      newly_rendered_image = draw_boxes(box, x0, y0, newly_rendered_image, prediction['class'])

  # WRITE
  save_with_bbox_renders(newly_rendered_image)

/content/inferred_videos
/content/inferred_videos
{'x': 754.5, 'y': 537.0, 'width': 15, 'height': 42, 'class': 'raquette_noire_Flore', 'confidence': 0.63, 'image_path': '/content/videos_to_infer/0001.png', 'prediction_type': 'ObjectDetectionModel'}
{'x': 709.5, 'y': 573.0, 'width': 15, 'height': 36, 'class': 'raquette_noire_Flore', 'confidence': 0.61, 'image_path': '/content/videos_to_infer/0006.png', 'prediction_type': 'ObjectDetectionModel'}
{'x': 780.0, 'y': 667.5, 'width': 30, 'height': 57, 'class': 'raquette_noire_Flore', 'confidence': 0.637, 'image_path': '/content/videos_to_infer/0028.png', 'prediction_type': 'ObjectDetectionModel'}
{'x': 792.0, 'y': 664.5, 'width': 30, 'height': 57, 'class': 'raquette_noire_Flore', 'confidence': 0.591, 'image_path': '/content/videos_to_infer/0029.png', 'prediction_type': 'ObjectDetectionModel'}
{'x': 796.5, 'y': 658.5, 'width': 27, 'height': 51, 'class': 'raquette_noire_Flore', 'confidence': 0.55, 'image_path': '/content/videos_to_infer/0030.pn

You must line up the starting file of your video (i.e. the image you want to start with) to the FFMPEG wildcard syntax.

For example, if you video file is named `out0001.png` use `y%04d.png`.

If your file is named `/content/inferred_videos/Pi_test_video.mov_out0001.png` use `/content/inferred_videos/Pi_test_video.mov_outy%04d.png`.



In [119]:
# stich images together into video
!pwd
!ffmpeg -r 10 -f image2 -s 1920x1080 -i /content/inferred_videos/%04d.png -vcodec libx264 -crf 25  -pix_fmt yuv420p test.mp4


/content/inferred_videos
ffmpeg version 4.3.2-0york0~18.04 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version='0york0~18.04' --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtw

- TODO: add garbage collector instructions here for repeat testing

In [ ]:
# CAUTION - deletes all data from the `inferred_videos` and `videos_to_infer` directories
# use this code block between runs for a fresh start between videos.
%cd /content/
!rm -r inferred_videos
!rm -r videos_to_infer
!mkdir inferred_videos
!mkdir videos_to_infer


- TODO - util features such as print list of all unique classe?
- TODO - update docs in notebooks
- TODO - update test cases

# Tested Cases

- no images exist
- image has no detections
- image has no target_detection instance
- image has less object count
- image has less class count
- prediction falls into confidence range
- prediction outside of confidence range
- prediction has less than box req
- prediction has greater than box req
- prediction doesn't match target_detection name
- first prediction meets requirements for upload
- last prediction meets requirements for upload
- all similarities match too high, even when images look drastically different